In [1]:
%cd mmdetection

# !pip install -q --upgrade wandb
# !pip3 install openmim
# !mim install mmcv-full
# !pip install -e .

from mmdet.apis import init_detector, inference_detector, show_result_pyplot
import mmcv
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets.builder import DATASETS
from mmdet.datasets.coco import CocoDataset,CustomDataset
from mmdet.apis import set_random_seed
import os.path as osp
import os
import numpy as np

import wandb
print(wandb.__version__)

/aiffel/aiffel/aimmo/mmdetection


/opt/conda/lib/python3.9/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


0.13.6


In [2]:
@DATASETS.register_module(force=True)
class Aimmo(CustomDataset):
    CLASSES = ('car', 'truck', 'bus','pedestrian')
    def load_annotations(self, ann_file):
        cat2label = {k:i for i, k in enumerate(self.CLASSES)}
        image_list = mmcv.list_from_file(self.ann_file)
        data_infos = []
    
        for image_id in image_list:
            filename = os.path.splitext(image_id)[0]#os.path.splitext('{0:}/{1:}'.format(img_prefix, image_id))[0]
            #image = cv2.imread(filename)
            data_info = {'filename': filename,
                         'width': 1920, 
                         'height': 1024}
            label_prefix = self.img_prefix.replace('image', 'middle_texts')
            lines = mmcv.list_from_file(osp.join(label_prefix, str(image_id)))

            content = [line.strip().split(' ') for line in lines]
            bbox_names = [x[0] for x in content]
            bboxes = [ [float(info) for info in x[1:5]] for x in content]

            gt_bboxes = []
            gt_labels = []
            gt_bboxes_ignore = []
            gt_labels_ignore = []

            for bbox_name, bbox in zip(bbox_names, bboxes):
                if bbox_name in cat2label:
                    gt_bboxes.append(bbox)
                    gt_labels.append(cat2label[bbox_name])
                else:
                    gt_bboxes_ignore.append(bbox)
                    gt_labels_ignore.append(-1)
            
            data_anno = {
              'bboxes': np.array(gt_bboxes, dtype=np.float32).reshape(-1, 4),
              'labels': np.array(gt_labels, dtype=np.compat.long),
              'bboxes_ignore': np.array(gt_bboxes_ignore, dtype=np.float32).reshape(-1, 4),
              'labels_ignore': np.array(gt_labels_ignore, dtype=np.compat.long)
            }
            data_info.update(ann=data_anno)
            data_infos.append(data_info)
        return data_infos

In [3]:
config = 'configs/yolox/yolox_l_8x8_300e_coco.py'
cfg = Config.fromfile(config)


cfg.runner['max_epochs'] = 12

cfg.log_config['interval'] = 1
cfg.model.bbox_head['num_classes'] = 4

cfg.checkpoint_config.interval =1 
cfg.log_config.interval = 10
cfg.log_config.hooks = [
    dict(type='TextLoggerHook'),
    dict(type='WandbLoggerHook',
         init_kwargs={'project': 'test-project', 'entity' : 'tesk-b', 'name' : 'yolo-x'})] 

cfg.load_from = 'checkpoints/yolox_l_8x8_300e_coco_20211126_140236-d3bd2b23.pth'
#cfg.resume_from=


cfg.dataset_type = 'Aimmo'
cfg.data_root = 'data/'

cfg.train_dataset.dataset.type = 'Aimmo'
cfg.train_dataset.dataset.ann_file = 'data/middle_train.txt'
cfg.train_dataset.dataset.img_prefix = 'data/image'



cfg.data.samples_per_gpu = 4
cfg.data.workers_per_gpu = 0

cfg.data.train.dataset.type = 'Aimmo'
cfg.data.train.dataset.ann_file = 'data/middle_train.txt'
cfg.data.train.dataset.img_prefix = 'data/image'

cfg.data.val.type = 'Aimmo'
cfg.data.val.ann_file = 'data/middle_val.txt'
cfg.data.val.img_prefix = 'data/image'

cfg.data.test.type = 'Aimmo'
cfg.data.test.ann_file = 'data/middle_sunday_test.txt'
cfg.data.test.img_prefix = 'data/test/image'

cfg.max_epochs = 12
cfg.num_last_epochs = 12
cfg.interval = 1

cfg.evaluation.save_best = 'mAP'
cfg.evaluation.interval = 1
cfg.evaluation.metric = 'mAP'

# Set seed thus the results are more reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)
cfg.data.persistent_workers = False
cfg.device = 'cuda'

cfg.work_dir = 'yolox'
print(cfg.pretty_text)

optimizer = dict(
    type='SGD',
    lr=0.01,
    momentum=0.9,
    weight_decay=0.0005,
    nesterov=True,
    paramwise_cfg=dict(norm_decay_mult=0.0, bias_decay_mult=0.0))
optimizer_config = dict(grad_clip=None)
lr_config = dict(
    policy='YOLOX',
    warmup='exp',
    by_epoch=False,
    warmup_by_epoch=True,
    warmup_ratio=1,
    warmup_iters=5,
    num_last_epochs=15,
    min_lr_ratio=0.05)
runner = dict(type='EpochBasedRunner', max_epochs=12)
checkpoint_config = dict(interval=1)
log_config = dict(
    interval=10,
    hooks=[
        dict(type='TextLoggerHook'),
        dict(
            type='WandbLoggerHook',
            init_kwargs=dict(
                project='test-project', entity='tesk-b', name='yolo-x'))
    ])
custom_hooks = [
    dict(type='YOLOXModeSwitchHook', num_last_epochs=15, priority=48),
    dict(type='SyncNormHook', num_last_epochs=15, interval=10, priority=48),
    dict(
        type='ExpMomentumEMAHook',
        resume_from=None,
        momentum=0.0001,

In [4]:
from mmdet.apis import multi_gpu_test, single_gpu_test
from mmcv.parallel import MMDataParallel, MMDistributedDataParallel
from mmdet.apis import inference_detector, init_detector, show_result_pyplot
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)


checkpoint_file = 'yolox/epoch_12.pth'  # 학습한 weight file
model_ckpt = init_detector(cfg, checkpoint_file, device='cuda:0')
model_ckpt = MMDataParallel(model_ckpt, device_ids=[0])

dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        # 반드시 아래 samples_per_gpu 인자값은 1로 설정
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

load checkpoint from local path: yolox/epoch_12.pth
The model and loaded state dict do not match exactly

unexpected key in source state_dict: ema_backbone_stem_conv_conv_weight, ema_backbone_stem_conv_bn_weight, ema_backbone_stem_conv_bn_bias, ema_backbone_stem_conv_bn_running_mean, ema_backbone_stem_conv_bn_running_var, ema_backbone_stem_conv_bn_num_batches_tracked, ema_backbone_stage1_0_conv_weight, ema_backbone_stage1_0_bn_weight, ema_backbone_stage1_0_bn_bias, ema_backbone_stage1_0_bn_running_mean, ema_backbone_stage1_0_bn_running_var, ema_backbone_stage1_0_bn_num_batches_tracked, ema_backbone_stage1_1_main_conv_conv_weight, ema_backbone_stage1_1_main_conv_bn_weight, ema_backbone_stage1_1_main_conv_bn_bias, ema_backbone_stage1_1_main_conv_bn_running_mean, ema_backbone_stage1_1_main_conv_bn_running_var, ema_backbone_stage1_1_main_conv_bn_num_batches_tracked, ema_backbone_stage1_1_short_conv_conv_weight, ema_backbone_stage1_1_short_conv_bn_weight, ema_backbone_stage1_1_short_conv_bn

/aiffel/aiffel/aimmo/mmdetection/mmdet/datasets/custom.py:181: UserWarning: CustomDataset does not support filtering empty gt images.
  warnings.warn(


In [5]:
outputs = single_gpu_test(model_ckpt, data_loader, True, 'my_result', 0.3)

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 393/393, 1.8 task/s, elapsed: 213s, ETA:     0s

In [6]:
dataset.evaluate(outputs,metric='mAP')


---------------iou_thr: 0.5---------------

+------------+------+-------+--------+-------+
| class      | gts  | dets  | recall | ap    |
+------------+------+-------+--------+-------+
| car        | 2057 | 18530 | 0.897  | 0.827 |
| truck      | 911  | 7489  | 0.855  | 0.719 |
| bus        | 319  | 2722  | 0.734  | 0.571 |
| pedestrian | 549  | 3700  | 0.570  | 0.385 |
+------------+------+-------+--------+-------+
| mAP        |      |       |        | 0.626 |
+------------+------+-------+--------+-------+


OrderedDict([('AP50', 0.626), ('mAP', 0.6258012652397156)])